# Vistas HTML
- Son ficheros HTML que sirven para representar los datos provistos por las acciones de los controladores.
- La sintaxis es un híbrido entre HTML y Python.
- El código HTML se escribe tal cual y el código Python se escapa entre dobles llaves.
    - Las expresiones van tras un igual:
        ```html
        <a href="{{=url_enlace}}">
        ```
    - {{=x}} inyecta el valor de x "escapado", a no ser que x sea un objeto de tipo XML.
    - Las sentencias van tal cual:
        ```html
        <ul>
        {{for item in list:}}
            <li>{{=item}}</li>
        {{pass}}
        </ul>
        ```
    - La indentación sigue las normas del HTML, no de Python. De modo que los bloques de código se cierran con "pass"
        ```python
        {{
        if i == 0:
        response.write('i is 0')       # no hay indentación
        else:
        response.write('i is not 0')   # no hay indentación
        pass                           # cierre del bloque if
        }}
        ```
    - Directivas Web2py:
        - extend: la vista que contiene esta directiva se inserta en la vista a la que extiende
            ```html
            {{extend 'layout.html'}}
            <h2>Formulario de alta</h2>
            {{=form}}
            ```
        - include:
            - include: la vista que contiene esta direcitiva inserta en esa posición la vista que la extiende
                Dada la vista "cuerpo.html":
                ```html
                <h1>Posts</h1>
                {{include}}
                ```
                Data la vista "posts.html":
                ```html
                {{extend 'cuerpo.html'}}
                <ul>
                {{for post in posts:}}
                    <li class="post">{{=post.autor}}<br>{{=post.texto}}</li>
                {{pass}}
                </ul>
                ```
                El resultado de renderizar "posts.html" se incrustará en "cuerpo.html" en la posición que ocupe la directiva {{include}}
            - include vista: la vista que contiene esta directiva inserta en esa posición el resultado de renderizar la vista incluida
                ```html
                {{include 'cabecera.html'}}
                <h2>Lista de elementos</h2>
                {{include 'lista-elementos.html'}}
                ```
        - block: permite nombrar bloques de las vistas para su posterior sustitución.
        
            layout.html
            ```html
            <html>
              <body>
                {{include}}
                <div class="cabecera">
                {{block lateral}}
                Contenido por defecto del bloque lateral (será sustituido)
                {{end}}
                </div>
              </body>
            </html>
            ```
            posts.html
            ```html
            {{extend 'layout.html'}}
            Este será mi contenido para la vista.
            {{block lateral}}
            Este es mi bloque lateral.
            {{end}}
            ```
        - super: permite que en un bloque se respete el contenido del padre al insertar otra vista
            layout.ntml
            ```html
            <html>
              <body>
                {{include}}
                <div class="sidebar">
                  {{block mysidebar}}
                    Contenido por defecto de mi sidebar (será sustituido)
                  {{end}}
                </div>
              </body>
            </html>
            ```
            
            vista.html
            ```html
            {{extend 'layout.html'}}
            Hola mundo
            {{block mysidebar}}
            {{super}}
            Nueva barra lateral
            {{end}}
            ```

            obtenemos
            ```html
            <html>
              <body>
                Hola mundo
                <div class="sidebar">
                    Contenido por defecto de mi sidebar (será sustituido)
                    Nueva barra lateral
                </div>
              </body>
            </html>
            ```
- La vista se renderiza secuencialmente.
- response.write() escribe en response.body
```html
<html><body>
{{for x in range(10):}}{{=x}}hello<br />{{pass}}
</body></html>
```
```python
# se traduce a
response.write("""<html><body>""", escape=False)
for x in range(10):
    response.write(x)
    response.write("""hello<br />""", escape=False)
response.write("""</body></html>""", escape=False)
```
- Si se produce un error en el renderizado, se muestra el código HTML generado.

### Estudio de layout.html y las vistas genéricas de la aplicación plantilla

**layout.html**

```html
<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]><!--> <html class="no-js" lang="{{=T.accepted_language or 'en'}}"> <!--<![endif]-->
  <head>
    <meta charset="utf-8">
    <!-- www.phpied.com/conditional-comments-block-downloads/ -->
    <!-- Always force latest IE rendering engine
         (even in intranet) & Chrome Frame
         Remove this if you use the .htaccess -->
    <meta http-equiv="X-UA-Compatible" content="IE=edge{{=not request.is_local and ',chrome=1' or ''}}">
    <!--  Mobile Viewport Fix
          j.mp/mobileviewport & davidbcalhoun.com/2010/viewport-metatag
          device-width: Occupy full width of the screen in its current orientation
          initial-scale = 1.0 retains dimensions instead of zooming out if page height > device height
          user-scalable = yes allows the user to zoom in -->
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{=response.title or request.application}}</title>
    <!-- http://dev.w3.org/html5/markup/meta.name.html -->
    <meta name="application-name" content="{{=request.application}}">
    <!-- Speaking of Google, don't forget to set your site up:
         http://google.com/webmasters -->
    <meta name="google-site-verification" content="">
    <!-- include stylesheets -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css"/>
    <link rel="stylesheet" href="{{=URL('static','css/bootstrap.min.css')}}"/>
    <link rel="stylesheet" href="{{=URL('static','css/web2py-bootstrap4.css')}}"/>
    <link rel="shortcut icon" href="{{=URL('static','images/favicon.ico')}}" type="image/x-icon">
    <link rel="apple-touch-icon" href="{{=URL('static','images/favicon.png')}}">
    <!-- All JavaScript at the bottom, except for Modernizr which enables
         HTML5 elements & feature detects -->
    <script src="{{=URL('static','js/modernizr-2.8.3.min.js')}}"></script>
    <!-- Favicons -->
    {{include 'web2py_ajax.html'}} <!-- this includes jquery.js, calendar.js/.css and web2py.js -->
    {{block head}}{{end}}
  </head>
  <body>
    <div class="w2p_flash alert alert-dismissable">{{=response.flash or ''}}</div>
    <!-- Navbar ======================================= -->
    <nav class="navbar navbar-light navbar-expand-md bg-faded bg-dark navbar-dark justify-content-center">
       <a href="http://web2py.com" class="navbar-brand d-flex w-50 mr-auto">web2py</a>
       <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNavDropdown" aria-controls="navbarNavDropdown" aria-expanded="false" aria-label="Toggle navigation">
         <span class="navbar-toggler-icon"></span>
       </button>
       <div class="navbar-collapse collapse w-100" id="navbarNavDropdown">
         <ul class="navbar-nav w-100 justify-content-center">
          {{for _item in response.menu or []:}}
          {{if len(_item)<4 or not _item[3]:}}
          <li class="nav-item {{if _item[1]:}}active{{pass}}">
            <a class="nav-link" href="{{=_item[2]}}">{{=_item[0]}}</a>
          </li>
          {{else:}}
          <li class="nav-item dropdown">
            <a class="nav-link dropdown-toggle" href="{{=_item[2]}}" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false">{{=_item[0]}}</a>
            <div class="dropdown-menu">
              {{for _subitem in _item[3]:}}
              <a class="dropdown-item" href="{{=_subitem[2]}}">{{=_subitem[0]}}</a>
              {{pass}}
            </div>
          </li>
          {{pass}}
          {{pass}}
        </ul>
         <form class="form-inline my-2 my-lg-0">
             <input class="form-control mr-sm-2" type="text" placeholder="Search">
         </form>
        {{if 'auth' in globals():}}
        <ul class="nav navbar-nav ml-auto w-100 justify-content-end">
          <li class="nav-item dropdown">
            <a class="nav-link dropdown-toggle" href="#" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false">
              {{if auth.user:}}{{=auth.user.first_name}}{{else:}}LOGIN{{pass}}
            </a>
            <div class="dropdown-menu dropdown-menu-right">
              {{if auth.user:}}
              <a class="dropdown-item" href="{{=URL('default','user/profile')}}">{{=T('Profile')}}</a>
              {{if 'change_password' not in auth.settings.actions_disabled:}}
              <a class="dropdown-item" href="{{=URL('default','user/change_password')}}">{{=T('Change Password')}}</a>
              {{pass}}
              <a class="dropdown-item" href="{{=URL('default','user/logout')}}">{{=T('Logout')}}</a>
              {{else:}}
              <a class="dropdown-item" href="{{=URL('default','user/login')}}">{{=T('Login')}}</a>
              {{if 'register' not in auth.settings.actions_disabled:}}
              <a class="dropdown-item" href="{{=URL('default','user/register')}}">{{=T('Sign up')}}</a>
              {{pass}}
              {{if 'retrieve_password' not in auth.settings.actions_disabled:}}
              <a class="dropdown-item" href="{{=URL('default','user/retrieve_password')}}">{{=T('Lost Password')}}</a>
              {{pass}}
              {{pass}}
            </div>
          </li>
        </ul>
        {{pass}}
      </div>
    </nav>

    <!-- Masthead ===================================== -->
    {{block header}}
    {{end}}
    <!-- Main ========================================= -->
    <!-- Begin page content -->
    <div class="container-fluid main-container">
      {{include}}
      {{=response.toolbar() if response.show_toolbar else ''}}
    </div>

    {{block footer}} <!-- this is default footer -->
    <footer class="footer container-fluid">
      <div class="row">
        <div class="col-md-12">
          <div class="copyright pull-left">{{=T('Copyright')}} &#169; {{=request.now.year}}</div>
          <div id="poweredBy" class="pull-right">
            {{=T('Powered by')}}
            <a href="http://www.web2py.com/">web2py</a>
          </div>
        </div>
      </div>
    </footer>
    {{end}}
    <!-- The javascript =============================== -->
    <script src="{{=URL('static','js/bootstrap.bundle.min.js')}}"></script>
    <script src="{{=URL('static','js/web2py-bootstrap4.js')}}"></script>
    {{block page_js}}{{end page_js}}
    {{if response.google_analytics_id:}}
    <!-- Analytics ==================================== -->
    <script src="{{=URL('static','js/analytics.min.js')}}"></script>
    <script type="text/javascript">
      analytics.initialize({
      'Google Analytics':{trackingId:'{{=response.google_analytics_id}}'}
      });
    </script>
    {{pass}}
  </body>
</html>
```

**generic.csv**
```python
{{"""Usage:

  def controller():
      return {"": db().select(db.thing.ALL)}

And then visit that controller with a .csv extention name
"""
}}{{if len(response._vars)==1:}}{{
# Not yet find a Python 2/3 compatible StringIO pattern,
# we avoid this solution http://web2py.com/books/default/chapter/29/10/services#CSV
# Here we buffer the entire csv file instead (it is your controller's job to limit the volume anyway),
# based on: http://web2py.com/books/default/chapter/29/06/the-database-abstraction-layer#CSV-one-Table-at-a-time-

content = response._vars[next(iter(response._vars))]
response.headers['Content-Type'] = 'application/vnd.ms-excel'
response.write(str(content), escape=False)
}}{{pass}}
```

**generic.html**
```python
{{extend 'layout.html'}}
{{"""

You should not modify this file. 
It is used as default when a view is not provided for your controllers

"""}}
<h2>{{=' '.join(x.capitalize() for x in request.function.split('_'))}}</h2>
{{if len(response._vars)==1:}}
{{=BEAUTIFY(response._vars[next(iter(response._vars))])}}
{{elif len(response._vars)>1:}}
{{=BEAUTIFY(response._vars)}}
{{pass}}
```

**generic.ics**
```python
{{
###
# response._vars contains the dictionary returned by the controller action
# Assuming something like:
#
# db.define_table('event',
#                 Field('title'),
#                 Field('start_datetime','datetime'),
#                 Field('stop_datetime','datetime'))
#   events = db(db.event).select()
#
# Aor this to work the action must return something like
#
#   dict(events=events, title='title',link=URL('action'),timeshift=0)
#
###
from gluon.serializers import ics}}{{=XML(ics(**response._vars))}}
```

**generic.json**
```python
{{from gluon.serializers import json}}{{=XML(json(response._vars))}}
```

**generic.jsonp**
```python
{{
###
# response._vars contains the dictionary returned by the controller action
###

# security check! This file is an example for a jsonp view.
# it is not safe to use as a generic.jsonp because of security implications.

if response.view == 'generic.jsonp':
   raise HTTP(501,'generic.jsonp disabled for security reasons')

try:
       from gluon.serializers import json
       result = "%s(%s)" % (request.vars['callback'], json(response._vars))
       response.write(result, escape=False)
       response.headers['Content-Type'] = 'application/jsonp'
except (TypeError, ValueError):
       raise HTTP(405, 'JSON serialization error')
except ImportError:
       raise HTTP(405, 'JSON not available')
except:
       raise HTTP(405, 'JSON error')
}}
```

**generic.load**
```python
{{'''
# License: Public Domain
# Author: Iceberg at 21cn dot com

With this generic.load file, you can use same function to serve two purposes.

= regular action
- ajax callback (when called with .load)

Example modified from http://www.web2py.com/AlterEgo/default/show/252:

def index():
    return dict(
		part1='hello world',
        	part2=LOAD(url=URL(r=request,f='auxiliary.load'),ajax=True))

def auxiliary():
    form=SQLFORM.factory(Field('name'))
    if form.accepts(request.vars):
        response.flash = 'ok' 
        return dict(message="Hello %s" % form.vars.name)
    return dict(form=form)

Notice:

- no need to set response.headers['web2py-response-flash']
- no need to return a string
even if the function is called via ajax.

'''}}{{if len(response._vars)==1:}}{{=response._vars[next(iter(response._vars))]}}{{else:}}{{=BEAUTIFY(response._vars)}}{{pass}}
```

**generic.map**
```python
{{"""
this is an example of usage of google map
the web2py action should be something like:

def map():
    return dict(
      googlemap_key='...',
      center_latitude = 41.878,
      center_longitude = -87.629,
      scale = 7,
      maker = lambda point: A(row.id,_href='...')
      points = db(db.point).select() where a points have latitute and longitude
    )

the corresponding views/defaut/map.html should be something like:

    \{\{extend 'layout.html'\}\}
    <center>\{\{include 'generic.map'\}\}</center>

"""}}
  <script src="http://maps.google.com/maps?file=api&amp;v=2&amp;key={{=googlemap_key}}" type="text/javascript"></script>
    <script type="text/javascript">
    //<![CDATA[
    function load() {
      if (GBrowserIsCompatible()) {
        var map = new GMap2(document.getElementById("map"));
        map.addControl(new GSmallMapControl());
        map.addControl(new GMapTypeControl());
        map.setCenter(new GLatLng({{=center_latitude}},
           {{=center_longitude}}), {{=scale}});
        // Create a base icon for all of our markers that specifies the
        // shadow, icon dimensions, etc.
        var baseIcon = new GIcon();
        baseIcon.shadow = "http://www.google.com/mapfiles/shadow50.png";
        baseIcon.iconSize = new GSize(20, 34);
        baseIcon.shadowSize = new GSize(37, 34);
        baseIcon.iconAnchor = new GPoint(9, 34);
        baseIcon.infoWindowAnchor = new GPoint(9, 2);
        baseIcon.infoShadowAnchor = new GPoint(18, 14);
        var blueIcon = new GIcon();
        blueIcon.image = "http://www.google.com/intl/en_us/mapfiles/ms/micons/blue-dot.png";
        blueIcon.shadow = "http://www.google.com/mapfiles/shadow50.png";
        blueIcon.iconSize = new GSize(37, 34);
        blueIcon.shadowSize = new GSize(37, 34);
        blueIcon.iconAnchor = new GPoint(9, 34);
        blueIcon.infoWindowAnchor = new GPoint(9, 2);
        blueIcon.infoShadowAnchor = new GPoint(18, 14);

        function createMarker(point, i, message) {
           // Set up our GMarkerOptions object
           if(i==0) markerOptions = { icon:blueIcon };
           else markerOptions= {}
           var marker = new GMarker(point, markerOptions);
           GEvent.addListener(marker, "click", function() {
             marker.openInfoWindowHtml(message);
           });
           return marker;
        }
        {{for point in points:}}{{if point.latitude and point.longitude:}}
          var point = new GLatLng({{=point.latitude}},{{=point.longitude}});
          map.addOverlay(createMarker(point, 0, 
            '{{=point.get('map_marker',maker(point))}}'));
        {{pass}}{{pass}}
     }
    }
    //]]>
    </script>
    <div id="map" style="width: 800px; height: 500px"></div>
    <script>load();</script>
```

**generic.pdf**
```python
{{
import os
from gluon.contrib.generics import pdf_from_html
filename = '%s/%s.html' % (request.controller,request.function)
if os.path.exists(os.path.join(request.folder,'views',filename)):
   html=response.render(filename)
else:
   html=BODY(BEAUTIFY(response._vars))
pass
=pdf_from_html(html)
}}
```

**generic.rss**
```python
{{
###
# response._vars contains the dictionary returned by the controller action
# for this to work the action must return something like
#
#   dict(title=...,link=...,description=...,created_on='...',items=...)
#
# items is a list of dictionaries each with title, link, description, pub_date.
###
from gluon.serializers import rss}}{{=XML(rss(response._vars))}}
```

**generic.xml**
```python
{{from gluon.serializers import xml}}{{=XML(xml(response._vars,quote=False))}}
```

**web2py_ajax.html**
```python
<script type="text/javascript"><!--
    // These variables are used by the web2py_ajax_init function in web2py_ajax.js (which is loaded below).
    {{=ASSIGNJS(
    w2p_ajax_confirm_message = T('Are you sure you want to delete this object?'),
    w2p_ajax_disable_with_message = T('Working...'),
    w2p_ajax_date_format = T('%Y-%m-%d'),
    w2p_ajax_datetime_format = T('%Y-%m-%d %H:%M:%S'),
    ajax_error_500 = T.M('An error occured, please [[reload %s]] the page') % URL(args=request.args, vars=request.get_vars)
    )}}
    //--></script>
{{
response.files.insert(0,URL('static','js/jquery.js'))
response.files.insert(1,URL('static','css/calendar.css'))
response.files.insert(2,URL('static','js/calendar.js'))
response.files.insert(3,URL('static','js/web2py.js'))
response.include_meta()
response.include_files()
}}
```

# Ayudantes HTML en [Web2py](http://www.web2py.com/books/default/chapter/29/05/the-views)
- Gracias a ellos evitamos la escritura de HTML "en crudo"
- Tienen el mismo nombre que el elemento HTML que representan, pero en mayúsculas

## HTML en Jupyter Notebook
Antes de analizar los ayudantes HTML de Web2py tenemos que ver cómo integrarlos en un notebook.

Jupyter Notebook puede renderizar HTML en sus celdas. Basta con crear una celda de tipo "code" cuya primera línea sea el comando "%%html". Las demás líneas de la celda serán el código HTML que deseemos renderizar.

In [1]:
%%html
<h1>HTML</h1>
<a href="http://www.google.com">Google</a>
<ul>
  <li>Item 1</li>
  <li>Item 2</li>
  <li>Item 3</li>
</ul>

## Entorno de trabajo Web2py + Jupyter Notebook
- Los ayudantes HTML de Web2py se definen en el fichero html.py del directorio gluon
- Jupyter Notebook dispone de herramientas para renderizar HTML

In [2]:
import sys
from IPython.display import display, Code as html_code


# path al directorio web2py
web2py_path = '../../curso/web2py'
sys.path.append(web2py_path)

# ayudantes HTML de Web2py
from gluon.html import *
# función de Jupyter que renderiza HTML
from IPython.core.display import HTML as IP_HTML

# función de apoyo para renderizar el HTML
# generado por Web2py
def w2p_render(html, verbose=False):
    'html -> (html, html_renderizado)'
    if verbose:
        display(html_code(str(html)))
    if 'Rows' in str(html.__class__):
        html = SQLTABLE(html)
    return IP_HTML(str(html))

## DIV
Firma:
```python
DIV(*componentes, **atributos)
```
Los componentes serán los elementos que vayan entre las etiquetas de apertura y cierre del elemento HTML.

Los atributos serán los atributos del elemento HTML. Sus nombres serán los mismos, pero con el prefijo "_".

Todos los ayudantes, excepto XML, derivan de DIV.

Tienen un par de métodos que permiten serializarlos:
```python
div.xml()
div.__str__()
```

In [4]:
# Veamos la documentación del ayudante DIV
DIV?

**Ejemplos**:

In [5]:
div = DIV('Hola', ' mundo')
# todos los ayudantes, excepto XML, derivan de DIV
print(div)

<div>Hola mundo</div>


In [5]:
div = DIV('Hola mundo', _style='background-color:silver;color:green;font-weight:bold')
# cuando trato de imprimirlo veo el código HTML generado
print(div)

<div style="background-color:silver;color:green;font-weight:bold">Hola mundo</div>


In [6]:
div = DIV('Hola mundo', _style='background-color:silver;color:green;font-weight:bold')
w2p_render(div, True)

<div style="background-color:silver;color:green;font-weight:bold">Hola mundo</div>

In [7]:
div = DIV('Hola mundo', DIV('Web2py', _class='alert alert-danger'), _id='info', _class='alert alert-info')
w2p_render(div, True)

<div class="alert alert-info" id="info">Hola mundo<div class="alert alert-danger">Web2py</div></div>

## DOM
Además de permitir la generación de HTML mediante código Python, son una representación del DOM en el lado del servidor.

Los componentes de los ayudantes HTML puden ser referenciados mediante su posición y los ayudantes HTML pueden considerarse listas de componentes.
```python
>>> a = DIV(DIV('Hola', ' mundo'), '!')
>>> print(a)
<div><div>Hola mundo</div>!</div>
>>> del a[1]   # '!'
>>> a.append(div('.'))
>>> a[0][0] = 'Vaya'   # 'Hola'
>>> print(a)
<div><div>Vaya mundo</div><div>.</div></div>
```

Los atributos de los ayudantes HTML pueden ser referenciados por su nombre y los ayudantes HTML pueden considerarse diccionarios de atributos.
```python
>>> a = DIV(DIV('Hola', ' mundo'), '!')
>>> a['_class'] = 'admiracion'
>>> a[0]['_class'] = 'saludo'
>>> print(a)
<div class="admiracion"><div class="saludo">Hola mundo</div>!</div>
```

Acceso a los todos los componentes y atributos:
```python
# lista de componentes
a.components
# atributos
a.attributes
```

Puede ser que los nombres de los atributos no sean válidos en Python, como por ejemplo "data-role". En esos casos podemos utilizar dos sintaxis distintas:
```python
>>> print DIV('text', data={'role': 'collapsible'})
<div data-role="collapsible">text</div>
>>> print DIV('text', **{'_data-role': 'collapsible'})
<div data-role="collapsible">text</div>
```

In [8]:
a = DIV(DIV('Hola', ' mundo'), '!')
a['_class'] = 'admiracion'
a[0]['_class'] = 'saludo'
print(a)
print('Componentes:\n\t', '\n\t'.join(map(str, a.components)))
print('Atributos:\n\t', '\n\t'.join(map(str, a.attributes)))

<div class="admiracion"><div class="saludo">Hola mundo</div>!</div>
Componentes:
	 <div class="saludo">Hola mundo</div>
	!
Atributos:
	 _class


## XML
Se usa para encapsular texto que no debe ser "escapado". Por ejemplo para renderizar una variable que contiene código HTML.

In [9]:
print(DIV("<b>hello</b>"))

<div>&lt;b&gt;hello&lt;/b&gt;</div>


In [10]:
print(DIV(XML("<b>hello</b>")))

<div><b>hello</b></div>


In [11]:
# el parámetro sanitize permite "asegurar" el HTML generado
print(DIV(XML('<script>alert("Peligro!")</script>', sanitize=True)))

<div>&lt;script&gt;&lt;/script&gt;</div>


In [12]:
print(XML('<script>alert("Peligro!")</script>',
          sanitize=True,
          permitted_tags=['a', 'b', 'blockquote', 'br/', 'i', 'li',
                          'ol', 'ul', 'p', 'cite', 'code', 'pre', 'img/'],
          allowed_attributes={'a':['href', 'title'],
                              'img':['src', 'alt'],
                              'blockquote':['type']}))

&lt;script&gt;&lt;/script&gt;


In [13]:
a = DIV(DIV('Hola', ' mundo'), '!')
a['_class'] = 'admiracion'
a[0]['_class'] = 'saludo'
print(a)

<div class="admiracion"><div class="saludo">Hola mundo</div>!</div>


## Lista de ayudantes HTML
Built-in:
- A
- ASSIGNJS
- B
- BODY
- BR
- CAT
- CENTER
- CODE
- COL
- COLGROUP
- DIV
- EM
- FIELDSET
- FORM
- H1, H2, H3, H4, H5, H6
- HEAD
- HTML
- XHTML
- HR
- I
- IFRAME
- IMG
- INPUT
- LABEL
- LEGEND
- LI
- META
- MARKMIN
- OBJECT
- OL
- ON
- OPTGROUP
- OPTION
- P
- PRE
- SCRIPT
- SELECT
- SPAN
- STYLE
- TABLE, TR, TD
- TBODY
- TEXTAREA
- TFOOT
- TH
- THEAD
- TITLE
- TR
- TT
- UL
- URL
- embed64
- xmlescape

Especiales
- TAG
- MENU

## BEAUTIFY
Se usa para construir una representación HTML de objetos.

In [14]:
w2p_render(BEAUTIFY({"Uno": ["Hola", XML("mundo")], "Dos": (1, 2)}), True)

<div><table><tr><td style="font-weight:bold;vertical-align:top;">Dos</td><td style="vertical-align:top;">:</td><td><div><table><tr><td><div>1</div></td></tr><tr><td><div>2</div></td></tr></table></div></td></tr><tr><td style="font-weight:bold;vertical-align:top;">Uno</td><td style="vertical-align:top;">:</td><td><div><table><tr><td><div>Hola</div></td></tr><tr><td><div>mundo</div></td></tr></table></div></td></tr></table></div>

## Server-side DOM
DIV y sus derivados tienen un par de atributos muy útiles para este punto:
- element devuelve el primer hijo que cumple una determinada condición o None si no lo hubiera
- elements devuelve la lista de todos los hijos que cumplen con dicha condición

La condición puede escribirse con tres sintaxis distintas, que pueden mezclarse:
- selectores jQuery
- valor exacto del atributo
- expresiones regulares

In [15]:
a = DIV(DIV(DIV('Hola', _id='uno', _class='miclase')))
print(a)
d = a.elements('div#uno')
d[0][0] = 'He cambiado'
print(a)

<div><div><div class="miclase" id="uno">Hola</div></div></div>
<div><div><div class="miclase" id="uno">He cambiado</div></div></div>


In [16]:
d1 = a.elements('#uno')
d2 = a.elements('div#uno')
d3 = a.elements('div[id=uno]')
d4 = a.elements('div', _id='uno')
assert d1 == d2 == d3 == d4

d5 = a.elements('.miclase')
d6 = a.elements('div.miclase')
d7 = a.elements('div[class=miclase]')
d8 = a.elements('div', _class='miclase')
assert d5 == d6 == d7 == d8

## Cajón de arena

In [17]:
attrs = {'_id': 'btn-1', '_class': 'btn btn-warning', '_href': 'http://www.google.com'}
content = ['Pincha', ' aquí', '!']
a = A(*content, **attrs)
w2p_render(a, True)

<a class="btn btn-warning" href="http://www.google.com" id="btn-1">Pincha aquí!</a>

In [18]:
form = FORM(LABEL('Nombre'), INPUT(_name='nombre'), INPUT(_type='submit'))
w2p_render(form, True)

<form action="#" enctype="multipart/form-data" method="post"><label>Nombre</label><input name="nombre" type="text" /><input type="submit" /></form>

In [19]:
button = BUTTON('Ok', _class='btn btn-danger')
w2p_render(button)

### Markmin

In [20]:
texto ='''
# Título
## Sección
### Subsección
**bold**
''italic''
~~strikeout~~
``verbatim``
``color con **bold**``:red
``varios colores``:color[blue:#ffff00]
http://google.com
[[**click** me #myanchor]]
[[click me [extra info] #myanchor popup]]	
[[alt-string for the image [the image title] http://www.web2py.com/examples/static/web2py_logo.png right 200px]]
- Dog
- Cat
- Mouse
+ Dog
+ Cat
+ Mouse
+ Dogs
 -- red
 -- brown
 -- black
+ Cats
 -- fluffy
 -- smooth
 -- bald
+ Mice
 -- small
 -- big
 -- huge
-----------------
**A**|**B**|**C**
=================
  0  |  0  |  X
  0  |  X  |  0
  X  |  0  |  0
=================
**D**|**F**|**G**
-----------------:abc[id]
-----
  This is a paragraph in a blockquote

  + item 1
  + item 2
  -- item 2.1
  -- item 2.2
  + item 3

  ---------
  0 | 0 | X
  0 | X | 0
  X | 0 | 0
  ---------:tableclass1
'''
mm = MARKMIN(texto)
w2p_render(mm)

A,B,C
0,0,X
0,X,0
X,0,0
D,F,G
0,0,X
0,X,0
X,0,0


## [Boostrap](https://getbootstrap.com/)
La aplicación plantilla de Web2py, welcome, utiliza Bootstrap como framework CSS, aunque el programador no está obligado a nada. Echaremos un vistazo rápido para que nos resulte más fácil entender las vistas creadas con dicho framework.

## [jQuery](https://jquery.com/)
Web2py utiliza jQuery como ayuda en la parte cliente, aunque el programador no está obligado a nada. Echaremos un vistazo rápido para que nos resulte más fácil entender cómo funciona Web2py en dicha parte.